In [1]:
from my_functions import*

In [2]:
file= uproot.open("/home/ruben/Escritorio/BachelorThesisRuben/Data/Muon_trees.root") #opening the Root file with Uproot 
MuonTree_ZeroBias=file["MuonTree_ZeroBias;1"]
MuonTree_Zmumu=file["MuonTree_Zmumu;1"]

In [3]:
#Prepare the data for the plots

#Choose the range of events to plot
nmin=0
nmax=1000

#Select quality 0 Z->mumu
Zmumu_pt=quality_selector(MuonTree_Zmumu["muon_quality"].array(),MuonTree_Zmumu["muon_pt"].array(),0)[nmin:nmax]
Zmumu_eta=quality_selector(MuonTree_Zmumu["muon_quality"].array(),MuonTree_Zmumu["muon_eta"].array(),0)[nmin:nmax]
Zmumu_phi=quality_selector(MuonTree_Zmumu["muon_quality"].array(),MuonTree_Zmumu["muon_phi"].array(),0)[nmin:nmax]

#And select the Z peak pairs
Zmumu_pt, Zmumu_eta, Zmumu_phi = get_all_Z_peak_pairs(Zmumu_pt,Zmumu_eta,Zmumu_phi)

#Select the ZeroBias data with energy cut
ZeroBias_eta=energy_cut(MuonTree_ZeroBias["muon_pt"].array(), MuonTree_ZeroBias["muon_eta"].array())[nmin:nmax]
ZeroBias_phi=energy_cut(MuonTree_ZeroBias["muon_pt"].array(), MuonTree_ZeroBias["muon_phi"].array())[nmin:nmax]
ZeroBias_pt=energy_cut(MuonTree_ZeroBias["muon_pt"].array(), MuonTree_ZeroBias["muon_pt"].array())[nmin:nmax]

quality_selector: Only 21.265 % of the data has been selected
quality_selector: Only 21.265 % of the data has been selected
quality_selector: Only 21.265 % of the data has been selected


100%|██████████| 1000/1000 [00:01<00:00, 969.15it/s]


energy_cut: Only 5.79% of the events have survived the cut with a lower cut of 14000 and no upper cut
energy_cut: Only 5.79% of the events have survived the cut with a lower cut of 14000 and no upper cut
energy_cut: Only 5.79% of the events have survived the cut with a lower cut of 14000 and no upper cut


In [4]:
res=muon_isolation_all_events(MuonTree_ZeroBias,ZeroBias_eta,ZeroBias_phi, 0.0, 0.4, [0,nmax])

In [5]:
jTower=MuonTree_Zmumu.arrays(["jTower_eta","jTower_calosource"],entry_start=100,entry_stop=101)
jTower_eta=jTower["jTower_eta"][0]
jTower_calosource=jTower["jTower_calosource"][0]

In [6]:
noise_cuts=jTower_assign_cuts(MuonTree_Zmumu)[0]
print(noise_cuts[0:20])

[750, 700, 750, 700, 600, 650, 650, 700, ..., 650, 650, 700, 600, 650, 650, 700]


In [7]:
import pandas as pd

# Crear DataFrame con nombres de columnas más legibles
df = pd.DataFrame({
    'CaloSource': jTower_calosource[18:30],
    'Eta': jTower_eta[18:30],
    'NoiseCut': noise_cuts[18:30]
})

# Mostrar tabla con título decorativo
print("\n📊 Primeros 20 Valores del jTower:")
print(df.to_string(index=False))  # Evita mostrar el índice



📊 Primeros 20 Valores del jTower:
 CaloSource      Eta  NoiseCut
          3 2.150000     650.0
          3 2.050000     700.0
          5 4.199668     350.0
          6 3.715290    1100.0
          5 3.407374     350.0
          6 3.341600    1100.0
          3 3.150000     350.0
          3 3.000000    1150.0
          3 2.450000     550.0
          3 2.800000    1250.0
          3 2.600000    1300.0
          3 2.450000     550.0


In [8]:
from IPython.display import display

display(df.style.set_caption("Example"))
df.style.hide(axis='index')



,CaloSource,Eta,NoiseCut
0,3,2.150000,650.000000
1,3,2.050000,700.000000
2,5,4.199668,350.000000
3,6,3.715290,1100.000000
4,5,3.407374,350.000000
5,6,3.341600,1100.000000
6,3,3.150000,350.000000
7,3,3.000000,1150.000000
8,3,2.450000,550.000000
9,3,2.800000,1250.000000


CaloSource,Eta,NoiseCut
3,2.150000,650.000000
3,2.050000,700.000000
5,4.199668,350.000000
6,3.715290,1100.000000
5,3.407374,350.000000
6,3.341600,1100.000000
3,3.150000,350.000000
3,3.000000,1150.000000
3,2.450000,550.000000
3,2.800000,1250.000000


In [13]:
from my_functions import *
file= uproot.open("/home/ruben/Escritorio/BachelorThesisRuben/Data/Muon_trees.root") #opening the Root file with Uproot 
MuonTree_ZeroBias=file["MuonTree_ZeroBias;1"]
MuonTree_Zmumu=file["MuonTree_Zmumu;1"]

jTower_cuts0 = ak.flatten(jTower_assign_cuts(MuonTree_ZeroBias, 0, 1))

for i in range(1, 20):
    jTower_cuts_i = ak.flatten(jTower_assign_cuts(MuonTree_ZeroBias, i, i + 1))
    if not np.allclose(ak.to_numpy(jTower_cuts_i), ak.to_numpy(jTower_cuts0), rtol=1e-5, atol=1e-8):
        print(f"Cuts differ at event {i}")


Cuts differ at event 1
Cuts differ at event 2
Cuts differ at event 3
Cuts differ at event 4
Cuts differ at event 5
Cuts differ at event 6
Cuts differ at event 7
Cuts differ at event 8
Cuts differ at event 9
Cuts differ at event 10
Cuts differ at event 11
Cuts differ at event 12
Cuts differ at event 13
Cuts differ at event 14
Cuts differ at event 15
Cuts differ at event 16
Cuts differ at event 17
Cuts differ at event 18
Cuts differ at event 19


In [4]:
def muon_isolation_all_events2(tree,muon_eta_all,muon_phi_all, lower_threshold, upper_threshold, event_range, batch_size=10000, get_mask=False):
    """
    This function computes muon isolation for all events in batches of a certain size to avoid crashing the computer.
    
    Inputs:
        tree: select a tree (MuonTree_ZeroBias or MuonTree_Zmumu)
        muon_eta/phi_all: awkward array containing the data of a Muon Tree (example: muon_eta_all=MuonTree_ZeroBias["LVLMuon_eta"].array())
        threshold: float that sets the maximum acceptable value for delta R (typically 0.4)
        event_range: tuple of [start_event, end_event]. To use the full dataset write [0,len(muon_eta_all)]
        batch_size: int that sets the number of events to process at once (shouldn't be larger than 5.000-10.000 for a PC with 8gb of RAM)
        get_mask: boolean that indicates if the mask should be returned or not, False by default
    Returns:
        List of isolated muon energies per event
    """
    #First compute the cuts for the jTower energies
    #jTower_cuts=jTower_assign_cuts(tree)

    start_event, end_event = event_range
    res = []
    masks=[]

    # Process in batches to avoid memory overload (crash due to lack of RAM)

    total_events = end_event - start_event

    #tqdm is used to get a progress bar that estimates the remaining time 
    #batch_stop is written like that to prevent going out of range
    for batch_start in tqdm(range(0, total_events, batch_size), desc="muon_isolation_all_events: Computing muon isolation", leave=False):
        batch_stop = min(batch_start + batch_size, total_events)

        # Load jTower data only for the current batch
        jTower = tree.arrays(
            ["jTower_eta", "jTower_phi", "jTower_et_MeV"],
            entry_start=start_event + batch_start,
            entry_stop=start_event + batch_stop
        )
        #assign eta, phi and et
        jTower_eta_batch = jTower["jTower_eta"]
        jTower_phi_batch = jTower["jTower_phi"]
        jTower_et_batch  = jTower["jTower_et_MeV"]

        #get the muon data for the current batch
        muon_eta_batch = muon_eta_all[batch_start:batch_stop]
        muon_phi_batch = muon_phi_all[batch_start:batch_stop]

        # Loop over events in a batch
        for i in range(len(muon_eta_batch)):
            event_index = start_event + batch_start + i
            jTower_cuts=ak.flatten(jTower_assign_cuts(tree,event_index,event_index+1))
            muon_eta_event = muon_eta_batch[i]
            muon_phi_event = muon_phi_batch[i]

            #Check if the event is empty before starting the calculations. This can reduce the computing time a lot if
            #our data includes many empty events 
            if len(muon_eta_event) == 0:
                res.append([])
            else:
                #If the event is not empty compute the isolation
                jTower_eta_event = jTower_eta_batch[i]
                jTower_phi_event = jTower_phi_batch[i]
                jTower_et_event  = jTower_et_batch[i]

                isol_event, mask= muon_isolation_one_event(
                    muon_eta_event, muon_phi_event,
                    jTower_eta_event, jTower_phi_event, jTower_et_event,
                    lower_threshold,upper_threshold,jTower_cuts,get_mask)
                #and append it to the result
                res.append(isol_event)
                #if get_mask is True, append the mask to the result
                if get_mask==True:
                    masks.append(mask)
    #if get_mask is True, return the result and the mask, otherwise just the result
    if get_mask==True:
        return res, masks
    else:
        return res

In [5]:
res1=muon_isolation_all_events(MuonTree_ZeroBias,ZeroBias_eta,ZeroBias_phi,0.0,0.4,[0,100])
res2=muon_isolation_all_events2(MuonTree_ZeroBias,ZeroBias_eta,ZeroBias_phi,0.0,0.4,[0,100])

In [6]:
res1==res2

True

In [7]:
res1

[[nan],
 [np.float32(1000.0)],
 [nan],
 [nan],
 [np.float32(1400.0)],
 [np.float32(2825.0)],
 [nan],
 [nan],
 [nan],
 [np.float32(6550.0)],
 [nan],
 [nan],
 [nan],
 [nan],
 [np.float32(800.0)],
 [nan],
 [np.float32(3250.0)],
 [nan],
 [nan],
 [np.float32(1375.0)],
 [nan],
 [nan],
 [np.float32(1675.0)],
 [np.float32(8325.0)],
 [nan],
 [np.float32(1200.0)],
 [nan],
 [nan],
 [np.float32(1025.0)],
 [np.float32(1325.0)],
 [nan],
 [nan],
 [np.float32(975.0)],
 [np.float32(1425.0)],
 [nan],
 [np.float32(975.0)],
 [nan],
 [np.float32(1125.0)],
 [nan],
 [nan],
 [nan],
 [nan],
 [np.float32(2025.0)],
 [nan],
 [nan],
 [nan],
 [np.float32(3100.0)],
 [nan],
 [np.float32(1250.0)],
 [nan],
 [nan],
 [nan],
 [np.float32(1000.0)],
 [np.float32(2475.0)],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [np.float32(1000.0)],
 [nan],
 [np.float32(2575.0)],
 [nan],
 [np.float32(625.0)],
 [np.float32(3875.0)],
 [nan],
 [nan],
 [nan, nan, nan],
 [np.float32(1150.0)],
 [nan],
 [np.float32(4975.0)

In [8]:
res2

[[nan],
 [np.float32(1000.0)],
 [nan],
 [nan],
 [np.float32(1400.0)],
 [np.float32(2825.0)],
 [nan],
 [nan],
 [nan],
 [np.float32(6550.0)],
 [nan],
 [nan],
 [nan],
 [nan],
 [np.float32(800.0)],
 [nan],
 [np.float32(3250.0)],
 [nan],
 [nan],
 [np.float32(1375.0)],
 [nan],
 [nan],
 [np.float32(1675.0)],
 [np.float32(8325.0)],
 [nan],
 [np.float32(1200.0)],
 [nan],
 [nan],
 [np.float32(1025.0)],
 [np.float32(1325.0)],
 [nan],
 [nan],
 [np.float32(975.0)],
 [np.float32(1425.0)],
 [nan],
 [np.float32(975.0)],
 [nan],
 [np.float32(1125.0)],
 [nan],
 [nan],
 [nan],
 [nan],
 [np.float32(2025.0)],
 [nan],
 [nan],
 [nan],
 [np.float32(3100.0)],
 [nan],
 [np.float32(1250.0)],
 [nan],
 [nan],
 [nan],
 [np.float32(1000.0)],
 [np.float32(2475.0)],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [np.float32(1000.0)],
 [nan],
 [np.float32(2575.0)],
 [nan],
 [np.float32(625.0)],
 [np.float32(3875.0)],
 [nan],
 [nan],
 [nan, nan, nan],
 [np.float32(1150.0)],
 [nan],
 [np.float32(4975.0)